# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sn

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
og_df = pd.read_csv('city_data.csv')
og_df = og_df.drop(labels='Unnamed: 0', axis = 1)
clean_df = og_df.dropna(how='any')
clean_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.29,-156.79,30.20,86.0,90.0,17.22,US,1.603596e+09
1,albany,42.60,-73.97,46.00,68.0,61.0,1.99,US,1.603596e+09
2,rikitea,-23.12,-134.97,74.88,82.0,97.0,20.71,PF,1.603596e+09
3,kandalaksha,67.16,32.41,27.88,97.0,92.0,4.36,RU,1.603596e+09
4,atuona,-9.80,-139.03,78.66,75.0,30.0,20.56,PF,1.603596e+09
...,...,...,...,...,...,...,...,...,...
624,channel-port aux basques,47.57,-59.14,50.18,96.0,100.0,8.46,CA,1.603596e+09
625,nhulunbuy,-12.23,136.77,87.80,58.0,40.0,10.29,AU,1.603596e+09
626,atbasar,51.80,68.33,39.60,92.0,100.0,19.84,KZ,1.603596e+09
627,mildura,-34.20,142.15,61.00,31.0,40.0,13.87,AU,1.603596e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = clean_df[["Lat", "Lon"]].astype(float)

# Convert to float and store
humid_values = clean_df["Humidity"].astype(float)


In [32]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_values, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
